# Product Processing For Shopify Template
Shopify allows for a a bulk upload of products, for which it provides a CSV template. This template is imported here, and our already existing product list is processed and put into the template. This is extremely useful, given the amount of products that we have.

In [5]:
import numpy as np
import pandas as pd
import random
import unidecode

In [6]:
template=pd.read_csv('product_template.csv')
template.head()

,Handle,Title,Body (HTML),Vendor,Type,Tags,Published,Option1 Name,Option1 Value,Option2 Name,...,Google Shopping / Custom Product,Google Shopping / Custom Label 0,Google Shopping / Custom Label 1,Google Shopping / Custom Label 2,Google Shopping / Custom Label 3,Google Shopping / Custom Label 4,Variant Image,Variant Weight Unit,Variant Tax Code,Cost per item
0,example-t-shirt,Example T-Shirt,NaN,Acme,Shirts,mens t-shirt example,True,Title,"Lithograph - Height: 9"" x Width: 12""",NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,g,NaN,NaN
1,example-t-shirt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Small,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,g,NaN,NaN
2,example-t-shirt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medium,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,g,NaN,NaN


An explanation for the variables in the template can be found at [Shopify's Admin Documentation Page](https://help.shopify.com/en/manual/products/import-export/using-csv#product-csv-file-format).

In [7]:
product_list=pd.read_csv('Product_List.csv')
product_list.head()

,Kategorie NEU,Sortierung,Produktbezeichnung,Produzent,Ländleherz,Saisonal \nerhältlich,Produkt Text
0,Alkoholfreie Gertänke & Sirup,Fruchtsaft,SUTTERLÜTY'S APFEL-BIRNENSAFT,"Mosterei Kramml, Lustenau",Us'm Ländle,NaN,Die Äpfel und Birnen für dieses reine Naturpro...
1,Alkoholfreie Gertänke & Sirup,Fruchtsaft,SUTTERLÜTY'S APFELSAFT,"Mosterei Kramml, Lustenau",Us'm Ländle,NaN,Der naturtrübe Apfelsaft von der Mosterei Kram...
2,Alkoholfreie Gertänke & Sirup,Fruchtsaft,SUTTERLÜTY'S APFELSAFT BOSKOOP,"Mosterei Kramml, Lustenau",Us'm Ländle,NaN,Der Boskoop Apfel – bei uns im Ländle auch Led...
3,Alkoholfreie Gertänke & Sirup,Fruchtsaft,SUTTERLÜTY'S APFELSAFT JONAGOLD,"Mosterei Kramml, Lustenau",Us'm Ländle,NaN,Jonagold Äpfel werden bei uns im Ländle seit d...
4,Alkoholfreie Gertänke & Sirup,Fruchtsaft gespritzt,SUTTERLÜTY'S APFELSAFT G'SPRITZT,"Mosterei Kramml, Lustenau",Us'm Ländle,NaN,Sutterlüty’s gespritzter Apfelsaft ist ein rei...


## 1. Feature Extraction
In this section I am writing  helper functions to extract features (product name, description) from this product list and modify and match them to the template.

In [44]:
"""
Transformation key table:
TEMPLATE <- PRODUCT LIST

Handle <- Produktbezeichnung w/o 'SUTTERLUTY'S', umlauts
Body <- Produkt Text
Title <- Produktbezeichnung w/o 'SUTTERLUTY'S'
Vendor <- Produzent
Type <- Sortierung
Tags <- TBD
Published <- FALSE (while in dev stage)
Option1 Name <- Title
Option1 Value <- Default Title
Other Option(s) <- TBD
Variant Grams <- 0 / TBD
Variant Inventory Policy <- deny
Variant Fulfillment Service <- manual
Variant Price <- rand() until we know prices

"""
def empty_template(template):
    return template[0:0]

def create_empty_row(template): 
    ncols=template.shape[1]
    arr=['' for i in range(ncols)]
    return arr

def fill_row(template,product_list,k):
    row=create_empty_row(template)
    #print(list(template.columns))
    for i in range(len(row)):
        j=list(template.columns)[i]
        if j=='Handle':
            handle=str(product_list['Produktbezeichnung'][k])
            handle=handle.lower()
            handle=unidecode.unidecode(handle) #remove umlauts for url handle
            handle=handle.replace("sutterluty's ","") #remove sutterluty
            handle=handle.replace("sutterluty’s","")
            handle=handle.replace(" / ","")
            handle=handle.replace(" ","-") #replace space with dash
            row[i]=handle
        elif j=="Title":
            handle=str(product_list['Produktbezeichnung'][k])
            handle=handle.lower()
            handle=handle.replace("sutterlüty's","")
            handle=handle.replace("sutterlüty’s","")
            handle=handle.capitalize()
            row[i]=handle
        elif j=='Body (HTML)':
            row[i]=product_list['Produkt Text'][k]
        elif j=='Vendor':
            row[i]=str(product_list['Produzent  '][k])
        elif j=='Type':
            row[i]=str(product_list['Sortierung'][k])
        elif j=='Published':
            row[i]='FALSE'
        elif j=='Option1 Name':
            row[i]='Title'
        elif j=='Option1 Value':
            row[i]='Default Title'
        elif j=='Variant Grams':
            row[i]=0
        elif j=='Variant Inventory Policy':
            row[i]=='deny'
        elif j=='Variant Fulfillment Service':
            row[i]='manual'
        elif j=='Variant Price':
            random.seed(123)
            row[i]=random.randint(1,10)
    #print(row)
    return row

In [45]:
shopify_product_list=empty_template(template) #create empty template
#print(shopify_product_list.shape)
count=0
for i in range(product_list.shape[0]): #fill in template row by row
    if type(product_list.iloc[i,0]) is str:
        row=np.array(fill_row(template,product_list,i))
        shopify_product_list.loc[count]=row
        count+=1
    else:
        continue #skip if NaN
shopify_product_list

,Handle,Title,Body (HTML),Vendor,Type,Tags,Published,Option1 Name,Option1 Value,Option2 Name,...,Google Shopping / Custom Product,Google Shopping / Custom Label 0,Google Shopping / Custom Label 1,Google Shopping / Custom Label 2,Google Shopping / Custom Label 3,Google Shopping / Custom Label 4,Variant Image,Variant Weight Unit,Variant Tax Code,Cost per item
0,apfel-birnensaft,apfel-birnensaft,Die Äpfel und Birnen für dieses reine Naturpro...,"Mosterei Kramml, Lustenau",Fruchtsaft,,FALSE,Title,Default Title,,...,,,,,,,,,,
1,apfelsaft,apfelsaft,Der naturtrübe Apfelsaft von der Mosterei Kram...,"Mosterei Kramml, Lustenau",Fruchtsaft,,FALSE,Title,Default Title,,...,,,,,,,,,,
2,apfelsaft-boskoop,apfelsaft boskoop,Der Boskoop Apfel – bei uns im Ländle auch Led...,"Mosterei Kramml, Lustenau",Fruchtsaft,,FALSE,Title,Default Title,,...,,,,,,,,,,
3,apfelsaft-jonagold,apfelsaft jonagold,Jonagold Äpfel werden bei uns im Ländle seit d...,"Mosterei Kramml, Lustenau",Fruchtsaft,,FALSE,Title,Default Title,,...,,,,,,,,,,
4,apfelsaft-g'spritzt,apfelsaft g'spritzt,Sutterlüty’s gespritzter Apfelsaft ist ein rei...,"Mosterei Kramml, Lustenau",Fruchtsaft gespritzt,,FALSE,Title,Default Title,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,geschenkpackle-klein,geschenkpäckle klein,Sutterlüty’s Bergkäse würzig 8 Monate gereift ...,nan,Geschenkpäckle,,FALSE,Title,Default Title,,...,,,,,,,,,,
167,geschenkpackle-mittel,geschenkpäckle mittel,Sutterlüty’s Bergkäse würzig 8 Monate gereift ...,nan,Geschenkpäckle,,FALSE,Title,Default Title,,...,,,,,,,,,,
168,geschenkpackle-gross,geschenkpäckle groß,"Sutterlüty’s Bio Salami 200 g, Sutterlüty’s Be...",nan,Geschenkpäckle,,FALSE,Title,Default Title,,...,,,,,,,,,,
169,b'sundrige-auswahl-individuell-bestuckbar\nmin...,b’sundrige auswahl individuell bestückbar / \...,"So individuell wie Ihre Mitarbeiter, Kunden un...",nan,Geschenkpäckle,,FALSE,Title,Default Title,,...,,,,,,,,,,


In [46]:
shopify_product_list.to_csv('shopify_product_list.csv', header=list(shopify_product_list.columns))

In [13]:
unidecode.unidecode("sutterlüty’s geschenkpäckle groß")

"sutterluty's geschenkpackle gross"

In [17]:
a=unidecode.unidecode("sutterlüty’s geschenkpäckle groß")
a=a.replace("sutterluty's ", "")
a

'geschenkpackle gross'

In [37]:
a="apfel-birnensaft"
a.capitalize()

'Apfel-birnensaft'